In [1]:
import os
import sys

In [2]:
IN_COLAB='google.colab' in sys.modules

In [3]:
if IN_COLAB:
    from google.colab import drive

    WORKING_FOLDER="/content/drive/MyDrive/unicamp/ia368v_dd/trabalho_final_UNICAMP-IR/artigo/unicamp_IR"

    drive.mount('/content/drive', force_remount=True)

    os.chdir(WORKING_FOLDER)
    
    CLUEWEB_PARTS_FOLDER="/content/drive/MyDrive/unicamp/ia368v_dd/trabalho_final_UNICAMP-IR/clueweb22-pt_10M_sample_fixed"
else:
    WORKING_FOLDER="/media/eduseiti/bigdata01/unicamp/ia368v_dd/unicamp_IR/"
    
    CLUEWEB_PARTS_FOLDER="../trabalho_final/clueweb22-pt_10M_sample_fixed"

In [4]:
import pandas as pd
import numpy as np
import glob

import pickle

In [5]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 500)

In [38]:
CLUEWEB22_MASSIVEWEB_1M_PT_BR_DATASET="clueweb22-pt_cleaned_massiveweb_1M_sample.tsv"

In [6]:
bm25_e5_df = pd.read_csv("tests/200-queries-passagens-1M-by-BM25-e5.tsv", sep="\t")

In [7]:
bm25_e5_df

,query,query_id,doc_id,score
0,Qual objetivo da Enel Brasil?,0,4439459,84.963623
1,Qual objetivo da Enel Brasil?,0,2640200,84.199814
2,Qual objetivo da Enel Brasil?,0,5239878,84.199814
3,Qual objetivo da Enel Brasil?,0,4306167,84.081741
4,Qual objetivo da Enel Brasil?,0,8368134,83.721863
...,...,...,...,...
199995,Quais são as principais religiões do Brasil?,199,3821763,71.740540
199996,Quais são as principais religiões do Brasil?,199,4403113,71.665215
199997,Quais são as principais religiões do Brasil?,199,8553141,70.562950
199998,Quais são as principais religiões do Brasil?,199,8369212,70.287613


In [8]:
bm25_e5_df.columns

Index(['query', 'query_id', 'doc_id', 'score'], dtype='object')

In [9]:
bm25_e5_df.shape

(200000, 4)

### Add query_id

In [10]:
all_queries_df = pd.read_csv("tests/all_queries.tsv", sep='\t')

In [11]:
all_queries_df

,query_id,query
0,0,Qual a maior característica da flora brasileira?
1,1,Qual a maior característica da fauna brasileira?
2,2,Por que os países Guiana e Suriname não são filiados a Conmebol?
3,3,Por que a Bolívia tem duas capitais?
4,4,Qual o que significa decolonialidade?
5,5,O que é sistema Swift?
6,6,Por que a seleção uruguaia ostenta 4 estrelas em seu escudo?
7,7,Qual a importância da Semana de Arte Moderna de 1922?
8,8,Por que Johann Sebastian Bach é considerado um revolucionário na história da música?
9,9,quais países europeus seguem o regime monarquista?


In [12]:
bm25_e5_df = bm25_e5_df[['query', 'doc_id', 'score']].merge(all_queries_df, left_on='query', right_on='query')

In [13]:
bm25_e5_df

,query,doc_id,score,query_id
0,Qual objetivo da Enel Brasil?,4439459,84.963623,100
1,Qual objetivo da Enel Brasil?,2640200,84.199814,100
2,Qual objetivo da Enel Brasil?,5239878,84.199814,100
3,Qual objetivo da Enel Brasil?,4306167,84.081741,100
4,Qual objetivo da Enel Brasil?,8368134,83.721863,100
...,...,...,...,...
199995,Quais são as principais religiões do Brasil?,3821763,71.740540,99
199996,Quais são as principais religiões do Brasil?,4403113,71.665215,99
199997,Quais são as principais religiões do Brasil?,8553141,70.562950,99
199998,Quais são as principais religiões do Brasil?,8369212,70.287613,99


In [14]:
bm25_e5_df['query_id'].unique().shape

(200,)

In [15]:
bm25_e5_df.groupby('query_id').count()['query'].unique()

array([1000])

In [16]:
which_queries = bm25_e5_df['query'].unique()

In [17]:
which_queries

array(['Qual objetivo da Enel Brasil?',
       'Qual a abordagem de mineração da moeda digital Chia (XCH)?',
       'Quais produtos são revendidos pela empresa Casa da Coifa?',
       'Em que ano surgiu a Universities and Colleges Admissions Service (UCAS)?',
       'Quais etapas para a inscrição pelo Colleges Admissions Service (UCAS)?',
       'Onde está localizada a Praça XV de Novembro?',
       'Quais os monumentos históricos situados na Praça XV de Novembro?',
       'Quando foi inaugurada a Estátua Equestre de Dom João VI?',
       'Mercado de trabalho para profissionais formados em Engenharia Química',
       'O que é o programa International Office da universidade São Judas?',
       'Quais profissionais podem fazer o curso de Práticas de Alfabetização?',
       'Quais os sintomas mais frequentes de metástases no cérebro?',
       'Tratamento para metástases cerebrais',
       'Fatores de risco para cancer de pele',
       'Quais os riscos nos ambientes hospitalares? ',
      

### Check which query is missing in the BM25 + e5 retrieval

In [18]:
all_queries = all_queries_df['query'].to_numpy()

In [19]:
all_queries.shape

(200,)

In [20]:
np.setdiff1d(all_queries, which_queries)

array([], dtype=object)

### Check if the existing queries in BM25 + e5 retrieval covers all the 50-sample queries

In [21]:
sampled_queries_df = pd.read_csv("tests/50-dataset-doccano-queries-passagens-by-BM25-mT5_with_query_id.tsv", sep="\t")

In [22]:
sampled_queries_df.head()

,passage-id,query,text,label,query_id
0,clueweb22-pt0001-14-16263_0,Onde está localizada a Praça XV de Novembro?,"Praça XV (Rio de Janeiro) – Wikipédia, a enciclopédia livre [ocultar] 5 línguas Praça XV (Rio de Janeiro) Origem: Wikipédia, a enciclopédia livre. Coordenadas: 22° 54′ 10″ S, 43° 10′ 24″ W Nota: Este artigo é sobre a praça da cidade do Rio de Janeiro. Para praças de outros lugares, veja Praça XV de Novembro. Esta página cita fontes, mas estas não cobrem todo o conteúdo. Ajude a inserir referências. Conteúdo não verificável poderá ser removido .— Encontre fontes: Google (notícias, livros e acadêmico)(Maio de 2019) A praça e as edificações das imediações. Trecho do Elevado da Perimetral que passava sobre a praça, em 2011. A Praça XV, também conhecida como Praça XV de Novembro, Praça Quinze de Novembro ou Praça Quinze, é uma praça situada no bairro do Centro, na Zona Central da cidade do Rio de Janeiro. Além de estar localizada na Rua Primeiro de Março, a praça integra a Orla Conde, um passeio público que margeia a Baía de Guanabara . A praça, que existe desde o século XVI, foi reinaugurada no dia 29 de maio de 2016 após ser revitalizada.[1] A revitalização da praça foi feita durante a gestão do prefeito Eduardo Paes no âmbito do Porto Maravilha, uma operação urbana que visa revitalizar a Zona Portuária do Rio de Janeiro.",0,105
1,clueweb22-pt0000-27-16948_2,Onde está localizada a Praça XV de Novembro?,"Rua XV de Novembro, 818 Centro, Pomerode, Santa Catarina 89107-000 Brasil Tela cheia O melhor por perto Restaurantes 100 dentro de 5 km Tarthurel Pizzaria 317 56 m $$ - $$$ • Italiana • Pizza Restaurante Torten Paradies 1.409 0.5 km $$ - $$$ • Docerias • Alemã • Brasileira Mon Cheri Bistro @ Casa Romena 106 0.7 km $$ - $$$ • Mediterrânea • Saudável • Leste europeia Veja tudo Atrações 24 dentro de 10 km Praça Jorge Lacerda 58 0.1 km Pontos de interesse Parque Municipal de Eventos 25 0.3 km Centros de convenção Parque Educativo Vila Encantada 1.057 0.9 km Parques Veja tudo Contribua Faça uma avaliação Carregue uma foto Avaliações Mais recente 4.5 722 avaliações Excelente 379 Muito boa 266 Razoável 68 Ruim 9 Horrível 1 Thais Leme, SP 161 contribuições Parada obrigatória para fotos mar de 2022 Quando eu via as fotos achava que era bem maior. O portal é muito bonito e tem um centro e informações turísticas. Estava no período da Osterfest, então estava todo com a decoração de Páscoa. Feita em 29 de abril de 2022 Esta avaliação representa a opinião subjetiva de um membro do Tripadvisor, e não da TripAdvisor LLC. daninoguei São Paulo, SP 2.143 contribuições MUITAS PESSOAS PARA TIRAR FOTOS mar de 2022 Portal turístico da cidade, local de parada para fotos com a representação da arquitetura local.",0,105
2,clueweb22-pt0001-14-16263_2,Onde está localizada a Praça XV de Novembro?,"A praça recebeu esse nome por homenagear o dia 15 de novembro de 1889, marcado pela Proclamação da República do Brasil. Nesse dia, foi instaurada a república federativa presidencialista no Brasil, em substituição à monarquia constitucional parlamentarista em vigor desde 1822. Índice 1 História 1.1 Período colonial 1.2 Século XIX 1.3 Século XX 1.4 Século XXI 2 Esculturas e Monumentos 2.1 Chafariz do Mestre Valentim 2.2 Estátua Equestre de Dom João VI 2.3 Estátua Equestre do General Osório 2.4 Estátua do Almirante Negro 2.5 Piskate 3 Transportes 3.1 Modais de transporte público 3.2 Elevado da Perimetral 3.3 Mergulhão da Praça XV 4 Pontos de interesse 5 Feiras 5.1 Feira de Antiguidades 5.2 Feirarte 6 Ver também 7 Referências 7.1 Bibliografia 8 Ligações externas História Período colonial Chafariz do Mestre Valentim, situado no centro da praça. A Praça XV hoje está localizada na região conhecida, nos primórdios da ocupação das terras da cidade, como Praia da Piaçaba. O local já teve diversas denominações: Largo do Terreiro da Polé;[4]Largo do Carmo;[5]Praça do Carmo; Terreiro do Paço; e Largo do Paço. Do século XVI até meados da década de 1770, com a construç

In [23]:
sampled_queries = sampled_queries_df['query'].unique()

In [24]:
np.setdiff1d(sampled_queries, which_queries)

array([], dtype=object)

### Filter BM25 + e5 passages only from the 50-sample queries

In [25]:
sampled_queries_list_df = sampled_queries_df.drop_duplicates('query')[['query_id', 'query']].reset_index(drop=True)

In [26]:
sampled_queries_list_df

,query_id,query
0,105,Onde está localizada a Praça XV de Novembro?
1,113,Fatores de risco para cancer de pele
2,115,Cuidados ao usar materiais perfurocortantes
3,117,Delirio versus alucinação
4,126,Quantas placas são formadas pelos materiais condutores?
5,127,O que é a associação em série de um capacitor?
6,128,Quando é o dia Mundial da Alimentação?
7,136,Qual foi a importância da usina de Volta Redonda RJ para a indusdustrialização brasileira?
8,147,Opções de Seguro da Essor Seguros
9,152,Quais os sinonimos de eficiência relacionado a competências?


In [28]:
bm25_e5_df.merge(sampled_queries_list_df[['query']], left_on="query", right_on="query")

,query,doc_id,score,query_id
0,Onde está localizada a Praça XV de Novembro?,7749553,83.469299,105
1,Onde está localizada a Praça XV de Novembro?,6267178,83.407135,105
2,Onde está localizada a Praça XV de Novembro?,5496753,83.109680,105
3,Onde está localizada a Praça XV de Novembro?,5849619,81.913399,105
4,Onde está localizada a Praça XV de Novembro?,7846756,81.616684,105
...,...,...,...,...
49995,Qual o critério para classificação para a Copa do Brasil?,3656264,72.902794,98
49996,Qual o critério para classificação para a Copa do Brasil?,3060430,72.594978,98
49997,Qual o critério para classificação para a Copa do Brasil?,4111273,72.391281,98
49998,Qual o critério para classificação para a Copa do Brasil?,8821978,71.515213,98


In [29]:
bm25_e5_only_sample_df = bm25_e5_df.merge(sampled_queries_list_df[['query']], left_on="query", right_on="query")

In [31]:
bm25_e5_only_sample_df

,query,doc_id,score,query_id
0,Onde está localizada a Praça XV de Novembro?,7749553,83.469299,105
1,Onde está localizada a Praça XV de Novembro?,6267178,83.407135,105
2,Onde está localizada a Praça XV de Novembro?,5496753,83.109680,105
3,Onde está localizada a Praça XV de Novembro?,5849619,81.913399,105
4,Onde está localizada a Praça XV de Novembro?,7846756,81.616684,105
...,...,...,...,...
49995,Qual o critério para classificação para a Copa do Brasil?,3656264,72.902794,98
49996,Qual o critério para classificação para a Copa do Brasil?,3060430,72.594978,98
49997,Qual o critério para classificação para a Copa do Brasil?,4111273,72.391281,98
49998,Qual o critério para classificação para a Copa do Brasil?,8821978,71.515213,98


In [32]:
bm25_e5_only_sample_df.columns = ['query', 'colbertx_id', 'score', 'query_id']

In [33]:
bm25_e5_only_sample_df

,query,colbertx_id,score,query_id
0,Onde está localizada a Praça XV de Novembro?,7749553,83.469299,105
1,Onde está localizada a Praça XV de Novembro?,6267178,83.407135,105
2,Onde está localizada a Praça XV de Novembro?,5496753,83.109680,105
3,Onde está localizada a Praça XV de Novembro?,5849619,81.913399,105
4,Onde está localizada a Praça XV de Novembro?,7846756,81.616684,105
...,...,...,...,...
49995,Qual o critério para classificação para a Copa do Brasil?,3656264,72.902794,98
49996,Qual o critério para classificação para a Copa do Brasil?,3060430,72.594978,98
49997,Qual o critério para classificação para a Copa do Brasil?,4111273,72.391281,98
49998,Qual o critério para classificação para a Copa do Brasil?,8821978,71.515213,98


### Now merge the passages clueweb ID into the retrieval results

In [34]:
id_conversion_df = pd.read_csv(os.path.join(CLUEWEB_PARTS_FOLDER, "clueweb22-pt_colbertx_to_clueweb_id.tsv"), sep="\t", names=['colbertx_id', 'clueweb_id'])

In [37]:
bm25_e5_only_sample_df = bm25_e5_only_sample_df.merge(id_conversion_df).sort_values(['query_id', 'score'], ascending=[True, False])

### Add passage texts

In [39]:
dataset_with_clueweb_id_df = pd.read_csv(os.path.join(CLUEWEB_PARTS_FOLDER, CLUEWEB22_MASSIVEWEB_1M_PT_BR_DATASET), sep="\t", header=None, names=['clueweb_id', 'passage', 'n_tokens', 'colbertx_id'])

In [40]:
dataset_with_clueweb_id_df

,clueweb_id,passage,n_tokens,colbertx_id
0,clueweb22-pt0000-00-00003_1,"Se você precisar de ajuda, visite o website nacional sobre a COVID-19 ou ligue para a linha de apoio à COVID-19 808 24 24 24 Perguntas mais frequentes Posso viajar entre Sintra e Cascais? Quais são as restrições de viagem em Cascais? Qual o número de telefone de apoio para a COVID 19 em Cascais? Preciso utilizar máscara facial no transporte público em Cascais? A prática do distanciamento social é compulsória em Cascais? O que eu devo fazer caso apresente sintomas da COVID-19 quando chegar em Cascais? Última atualização: 25 Abr 2022 Aplicam-se exceções, para detalhes completos: European Union. Estamos trabalhando ininterruptamente para lhe trazer as últimas informações de viagem relacionadas à COVID-19. Esta informação é compilada a partir de fontes oficiais. Ao melhor de nosso conhecimento, está correta de acordo com a última atualização. Visite Avisos de Viagem Rome2rio para ajuda geral. Perguntas & Respostas Qual a maneira mais econômica de ir de Sintra para Cascais? Qual a maneira mais rápida de ir de Sintra para Cascais? Existe um ônibus direto entre Sintra e Cascais? Qual a distância entre Sintra e Cascais? Como viajar de Sintra para Cascais sem carro? Qual a distância entre Sintra e Cascais? Aonde pegar ônibus de Sintra para Cascais?",361,5
1,clueweb22-pt0000-00-00003_2,"Qual a parada final de Sintra para Cascais ônibus? Posso dirigir de Sintra para Cascais? Quais são as opções de acomodação perto de Cascais? Abrir visualização de mapa Distância: 12.2 km Duração 33 min Quais empresas operam serviços entre Sintra, Portugal e Cascais, Portugal? Operadores de ônibus Scott URB FlixBus Outros Operadoras BlaBlaCar Táxi de Sintra para Cascais Mais Perguntas e Respostas Existe opções de transporte compartilhado entre Sintra e Cascais? Cascais, Portugal Cascais MHC é uma vila portuguesa, sede do município (concelho) homónimo, o qual é parte do distrito e da área metropolitana de Lisboa. Em 2016, totalizava habitantes distribuídos por uma área de. O município subdivide-se em quatro freguesias (Alcabideche, Carcavelos e Parede, Cascais e Estoril e São Domingos de Rana). Apesar do seu estatuto de vil ... Coisas para fazer em Cascais Quinta da Regaleira O Palácio da Regaleira é o edifício principal e o nome mais comum do palácio da Quinta da Regaleira. Também é designado Palácio do Monteiro dos Milhões, denominação esta associada à alcunha do seu primeiro proprietário foi António Augusto que foi distinguido pelo rei Dom Carlos I em 16 de agosto de 1904 como barão de Almeida. Casino Estoril",382,6
2,clueweb22-pt0000-00-00003_3,"O Casino Estoril fica localizado no Estoril, a 18 km de Lisboa e a 20 km do Aeroporto de Lisboa. É hoje o maior casino da Europa e é gerido pela empresa Estoril-Sol SGPS S.A.. Cabo da Roca O Cabo da Roca é o ponto mais ocidental de Portugal continental, assim como da Europa continental. Situa-se na freguesia de Colares, concelho de Sintra e distrito de Lisboa. O local é visitável, não até ao extremo mas até uma zona à altitude de 140 m. O cabo forma o extremo ocidental da Serra de Sintra, precipitando-se sobre o Oceano Atlântico. As suas coordenadas geográficas são N 38º46'51\"", W 9º30'2\"". Lugares para ficar em Cascais €226 InterContinental Cascais-Estoril, an IHG Hotel 4.8 Exceptional €180 Hotel Cascais Miragem 4.8 Exceptional €133 Grande Real Villa Italia 4.8 Exceptional Rome2rio facilita as viagens de Sintra a Cascais.",255,7
3,clueweb22-pt0000-00-00003_5,Sintra para Lisboa Sintra para Óbidos Leiria Portugal Sintra para Colares Portugal Sintra para Coimbra Sintra para Assafora Sintra para Valado dos Frades Sintra para Centro Colombo Sintra para Bombarral Sintra para Santarém Portugal Sintra para Santa Iria da Azóia Sintra para Salamanca Castela e Leão Espanha Sintra para Autódromo Internacional do Algarve Sintra para Alenquer Portugal Sintra para Vila Nova de Milfontes Sintra para Penha Longa Resort Sintra Sintra para Alverca do Ribat

In [42]:
bm25_e5_only_sample_df = bm25_e5_only_sample_df.merge(dataset_with_clueweb_id_df)

In [48]:
bm25_e5_only_sample_df = bm25_e5_only_sample_df.sort_values(['query_id', 'score'], ascending=[True, False]).reset_index(drop=True)

In [50]:
bm25_e5_only_sample_df.iloc[1999:2010]

,query,colbertx_id,score,query_id,clueweb_id,passage,n_tokens
1999,Por que os países Guiana e Suriname não são filiados a Conmebol?,850469,71.871689,2,clueweb22-pt0000-17-11832_0,"Associado - Dicio, Dicionário Online de Português associado Significado de Associado adjetivo Ligado por interesses comuns; sócio. Que participa de uma associação particular, organização ou clube esportivo; afiliado, membro, filiado. Parte de uma confederação, de uma coligação (pessoas, partidos, países). Relacionado com o que foi anteriormente mencionado: termos associados ao assunto principal. Que se conseguiu associar, unir, juntar, relacionar. substantivo masculino Indivíduo que é sócio ou que faz parte de uma sociedade: os associados ficam com 10% dos lucros da empresa. Etimologia (origem da palavra associado ). Particípio de associar. Sinônimos de Associado Associado é sinônimo de: agregado, confederado, ligado, unido, sócio, agrupado, afiliado, membro, filiado Definição de Associado Classe gramatical: adjetivo e substantivo masculino Flexão do verbo associar no: particípio Separação silábica: as-so-ci-a-do Plural: associados Feminino: associada Exemplos com a palavra associado Lennon também comentou o estilo de Bono já associado aos óculos. Folha de S.Paulo, 06/01/2013 O sobrepeso durante a gravidez está associado a problemas como diabetes, pressão alta e parto prematuro. Folha de S.Paulo, 18/05/2012",358
2000,quais países europeus seguem o regime monarquista?,760697,84.671425,9,clueweb22-pt0000-15-09528_8,"exprimiu-se nas idéias de Paine e Franklin, resultando na luta contra o Despotismo Esclarecido da Coroa britânica no século XVIII. e) No caso da Prússia, não houve ilustração Política, nem Abolicionismo ilustrado, já que a nação alemã não se havia unificado politicamente na Época Moderna. 14. (UFJF MG/1998) O processo de centralização do poder real e unificação territorial, associado ao crescente controle do Estado sobre a economia, conduziu a maior parte dos países europeus à constituição de monarquias nacionais absolutistas. Esse processo teve, contudo, importantes variações regionais. Enumere a 2ª coluna de acordo com a 1ª e, em seguida, marque a alternativa CORRETA: (1) Portugal (2) Espanha (3) França (4) Inglaterra (5) Alemanha (__) o processo de centralização do poder real e unificação territorial foi dificultado devido à força da nobreza feudal luterana; (__) a unidade política foi fraca devido à persistência de diversidades regionais e pluralidade cultural e religiosa, que pode ser exemplificada pela Guerra de Reconquista; (__) foi o primeiro país europeu a se unificar territorial e politicamente, em grande parte devido ao apoio da nobreza e da burguesia ao rei, (__) o processo de centralização real e unificação",346
2001,quais países europeus seguem o regime monarquista?,4102030,84.644524,9,clueweb22-pt0000-86-06698_41,"Essas palavras, pronunciadas por Rosencrantz, personagem de um drama teatral de William Shakespeare, aludem a) ao absolutismo monárquico, regime político predominante nos países europeus da Idade Moderna. b) à monarquia parlamentarista, na qual os poderes políticos derivam do consentimento popular. c) ao poder mais simbólico do que verdadeiro do rei, expresso pela máxima “o rei reina, mas não governa”. d) à oposição dos Estados europeus à ascensão da burguesia e à emergência das revoluções democráticas. e) à decapitação do monarca inglês pelo Parlamento durante as Revoluções Puritana e Gloriosa. resposta da questão 41:[A] Comentário da questão: Dado o contexto da referida obra, pode-se afirmar que o enunciado alude ao absolutismo monárquico e aos problemas relativos à sucessão no interior daquele referido regime. 42. (UFPR-2009) Nos séculos XVI e XVII prevaleceram na Europa Ocidental sistemas de organização do poder genericamente denominados por Antigo Regime. Assinale a alternativa que apresenta um conjunto de elementos INEXISTENTES no Antigo Regime. a) Absolutismo. b) Taylorismo. c) Mercantilismo. d) Sistema Colon

In [51]:
bm25_e5_only_sample_df.rename(columns={"clueweb_id": "passage_id"}).to_csv(os.path.join("tests", "bm25_e5_1M_docs_fixed_000_999_sampled_50_queries.tsv"), sep="\t", index=False)

In [52]:
bm25_e5_only_sample_df.shape

(50000, 7)

In [54]:
bm25_e5_only_sample_top_10_df = bm25_e5_only_sample_df.groupby('query_id').head(10).reset_index(drop=True)

In [55]:
bm25_e5_only_sample_top_10_df

,query,colbertx_id,score,query_id,clueweb_id,passage,n_tokens
0,Qual a maior característica da fauna brasileira?,4875435,84.761627,1,clueweb22-pt0001-01-18874_0,"Conheça 11 lindos animais típicos da fauna brasileira / Incrível Conheça 11 lindos animais típicos da fauna brasileira 4--4 Compartilhe no Facebook Compartilhe no Facebook Compartilhe no Pinterest Que o Brasil é gigante, todos nós sabemos e até já demos aqui exemplos chocantes sobre isso. Mas um outro exemplo dessa grandiosidade é quando sabemos que o país abriga 17% das espécies de aves e 10% das de anfíbios e mamíferos de todo o mundo. Neste post, o Incrível.club vai te mostrar uma lista de animais típicos do país. Mico-leão dourado © Nadine Doerle / Pixabay Este macaco “ruivo” ficou famoso ao tornar-se um símbolo de espécies brasileiras em extinção. O cuidado para a preservação da espécie, que é nativa da Mata Atlântica e aparece na nota de 20 reais, começou nos anos 1970 e se intensificou duas décadas depois. Embora a situação tenha melhorado, o risco de extinção ainda existe. Tamanduá-bandeira © joelfotos / Pixabay Este gigante que ama comer formigas — pode devorar até 30 mil por dia, inclusive cupins — é também um forte: com seus 2,20 metros, consegue até brigar com uma onça pintada. E vencer. Embora ele exista originalmente nas três Américas, já foi extinto em países como Estados Unidos e em Estados brasileiros como Espírito Santo e Rio e Janeiro. Onça pintada",381
1,Qual a maior característica da fauna brasileira?,2605802,84.615746,1,clueweb22-pt0000-55-02859_0,"Fauna e Flora: : entenda o que são e quais as diferenças (com exemplos) - Toda Matéria Biologia Fauna e Flora A fauna é o grupo de animais de determinada região. Já a flora é a representação das espécies vegetais. Quando se fala em fauna e flora brasileira, estão sendo referidos a totalidade de animais e plantas que tem seu habitat em um dos biomas existentes no Brasil. No Brasil, existem seis grandes biomas: cerrado, caatinga, mata atlântica, pampas, pantanal e a amazônia. Ou seja, cada comunidade apresenta características específicas e sua própria biodiversidade. Os animais (fauna) e as plantas (flora) variam de acordo com o meio em que se encontram. O que é fauna? A fauna representa a comunidade de espécies animais que habitam um ambiente específico ou alargado. Desse modo, o termo pode ser empregado em relação a um bioma específico, por exemplo, fauna do cerrado ou, em geral, como fauna brasileira ou mundial. Também pode possuir um sentido ainda mais amplo, relativo a um período: por exemplo, \""fauna do período jurássico\"". Em outras palavras, a representação do coletivo de animais de determinado local ou período é chamada fauna.",314
2,Qual a maior característica da fauna brasileira?,7028401,84.480431,1,clueweb22-pt0001-46-05303_17,"http://www.klimanaturali.org/2014/03/biomas-e-fitofisionomias-brasileiros.html 8/17 f15/03/2018 BIOMAS E FITOFISIONOMIAS BRASILEIROS: CARACTERÍSTICAS E EVOLUÇÃO DA FAUNA E FLORA - Klima Naturali™ colonização no Brasil e os ciclos de desenvolvimento do país SITE  BRASIL  MUNDO  CIÊNCIAS  GERAL  levaram o homem a ocupar e destruir parte desse espaço. Fauna e Flora Rica em diversidade de espécies, a Mata Atlântica está entre Pesquisa no Site...  as cinco regiões do mundo com maior número de espécies endêmicas. Os animais podem ser considerados generalistas ou especialistas. Os generalistas apresentam hábitos alimentares variados, alta taxa de crescimento e dispersão; vivem em áreas de vegetação aberta e secundária, tolerantes e capazes de aproveitar diferentes recursos oferecidos pelo meio ambiente. Os animais especialistas são mais exigentes em relação aos habitats nos quais vivem, com dieta especí ca, uma alteração no meio ambiente exige dos animais especialistas a procura de novos habitats. Grupos da Mata Atlântica Mamíferos – A Mata Atlântica possui cerca de 250 espécies de mamíferos, das quais 55 são endêmicas. Os mamíferos são os",347
3,Qual a maior caracter

In [56]:
bm25_e5_only_sample_top_10_df.rename(columns={"clueweb_id": "passage_id"}).to_csv(os.path.join("tests", "bm25_e5_1M_docs_fixed_000_009_sampled_50_queries.tsv"), sep="\t", index=False)

### Check execution log

In [57]:
with open("tests/execution_with_reverse_rev_2.1_sampled_50_queries_gpt-4-1106-preview_20240125_02.pkl", "rb") as input_file:
    logs = pickle.load(input_file)

In [58]:
logs.keys()

dict_keys(['args', 'evaluation_cache', 'reverse_mapping'])

In [59]:
list(logs['evaluation_cache'].keys())[0]

'Onde está localizada a Praça XV de Novembro?_clueweb22-pt0000-44-08794_4'

In [60]:
bm25_e5_only_sample_top_10_df['query'].unique()

array(['Qual a maior característica da fauna brasileira?',
       'Por que os países Guiana e Suriname não são filiados a Conmebol?',
       'quais países europeus seguem o regime monarquista?',
       'quais os critérios de definição dos monumentos intitulados maravilhas do mundo moderno?',
       'Como o Brasil reagiu a epidemia de AIDS no fim do século XX?',
       'Qual a maior torcida de futebol do Brasil?',
       'Quando se realizou o plebiscito popular para definir o sistema político do Brasil?',
       'Por que a legislação de um país é tão importante?',
       'Qual o nosso atual sistema de governo?',
       'Qual o sistema de governo da África do Sul?',
       'Por que dividir um país em estados?',
       'Como podemos classificar o relevo brasileiro?',
       'Existem vantagens ao definir uma moeda única?',
       'Como transformar uma cidade pacata em um polo turístico?',
       'Quais são os melhores parques nacionais de Portugal?',
       'Qual a melhor cidade para ficar

In [61]:
already_analized = 0

for i, row in bm25_e5_only_sample_top_10_df.iterrows():
    if (row['query'] + "_" + row['clueweb_id']) in logs['evaluation_cache']:
        print("Already analized...")

        already_analized += 1

Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...


In [62]:
already_analized

342

In [63]:
500 - 342

158